# 2.3 Special Columns (Ascending Keys)


As per [https://docs.microsoft.com/en-us/azure/synapse-analytics/sql/develop-tables-statistics](https://docs.microsoft.com/en-us/azure/synapse-analytics/sql/develop-tables-statistics)

_Consider updating "ascending key" columns such as transaction dates more frequently because these values won't be included in the statistics histogram._

As a demo lets show a table with a date missing and perfect stats. Also introducing the StatsOptimize SProc to automate updating of Stats

In [22]:
delete from  [dbo].[FactFinance100m_nostats] WHERE DateKey> 20130000

exec  dbo.[StatsOptimize]  @Tables='FactFinance100m_nostats'
, @StatisticsModificationLevel=null
, @StatisticsSample=100 
, @OnlyModifiedStatistics=null
, @DeleteOverlappingStats=null
, @TimeLimit=null 
, @Execute='Y'


(10000000 rows affected)

Date and time: 2022-07-07 20:39:49

Server: aw-dev

Version: 10.0.15665.0

Edition: SQL Azure

Parameters: @Tables= 'FactFinance100m_nostats', @OnlyModifiedStatistics = 'Y', @StatisticsModificationLevel = NULL, @StatisticsSample = 100, @DeleteOverlappingStats = N, @TimeLimit = NULL, @Execute = Y

Version: Microsoft Azure SQL Data Warehouse - 10.0.15665.0 Jul 6 2022 10:30:07 Copyright (c) Microsoft Corporation

Source: https://github.com/ProdataSQL/SynapseTools/tree/main/SqlPools/Maintenance

Date and time: 2022-07-07 20:39:55

Total execution time: 00:00:11.954

In [28]:
/* The Stats Histogram is now mising latest date (while being perfect)*/
DBCC SHOW_STATISTICS ('FactFinance100m_nostats',_WA_Sys_00000001_76A2557D)

(1 row affected)

(1 row affected)

(27 rows affected)

Total execution time: 00:00:00.111

Name,Updated,Rows,Rows Sampled,Steps,Density,Average key length,String Index,Filter Expression,Unfiltered Rows,Persisted Sample Percent
_WA_Sys_00000001_76A2557D,Jul 7 2022 8:37PM,67403926,67403926,27,0.037037037,4,NO,NULL,67403926,0


All density,Average Length,Columns
0.037059914,4,DateKey


RANGE_HI_KEY,RANGE_ROWS,EQ_ROWS,DISTINCT_RANGE_ROWS,AVG_RANGE_ROWS
20101229,0,4318566,0,1
20110129,0,4364350,0,1
20110301,0,4357974,0,1
20110331,0,3807165,0,1
20110501,0,3409494,0,1
20110531,0,3667331,0,1
20110701,0,3336511,0,1
20110801,0,3064507,0,1
20110829,0,3118080,0,1
20110929,0,2602791,0,1


In [24]:
/* Lets query Rows in one day. Query is very fast */


select d.[FiscalYear], d.DayOfWeek, sum(Amount) as Amount, count(*) as Rows  from 
FactFinance100m_nostats f 
INNER JOIN DimDate d on d.[DateKey]=f.DateKey 
WHERE d.DateKey=20120129
GROUP BY  d.[FiscalYear], d.DayOfWeek

(1 row affected)

Total execution time: 00:00:00.882

FiscalYear,DayOfWeek,Amount,Rows
2011,Sunday,105775666467.75511,2676081


In [25]:
/* Lets insert 10 million rows into today to simulate a load */
INSERT INTO FactFinance100m_nostats ([AccountKey], [ScenarioKey], [DepartmentGroupKey], [DateKey], [OrganizationKey], [Amount], [Date], [LineageKey])
SELECT  TOP 10000000 [AccountKey], [ScenarioKey], [DepartmentGroupKey], [DateKey], [OrganizationKey], [Amount], [Date], [LineageKey]  
FROM [dbo].[FactFinance1b]  WHERE dateKey >20130000

(10000000 rows affected)

Total execution time: 00:00:15.727

In [26]:
/* Lets try same query now. Current Date will have no stats in histogram */
select d.[FiscalYear], d.DayOfWeek, sum(Amount) as Amount, count(*) as Rows  from 
FactFinance100m_nostats f 
INNER JOIN DimDate d on d.[DateKey]=f.DateKey 
WHERE d.DateKey=20130829
GROUP BY  d.[FiscalYear], d.DayOfWeek

(1 row affected)

Total execution time: 00:00:01.130

FiscalYear,DayOfWeek,Amount,Rows
2013,Thursday,41245969830.65645,1172356


In [27]:
/* Lets query an older date which has stats */
select d.[FiscalYear], d.DayOfWeek, sum(Amount) as Amount, count(*) as Rows  from 
FactFinance100m_nostats f 
INNER JOIN DimDate d on d.[DateKey]=f.DateKey 
WHERE d.DateKey=20120630
GROUP BY  d.[FiscalYear], d.DayOfWeek


(1 row affected)

Total execution time: 00:00:00.348

FiscalYear,DayOfWeek,Amount,Rows
2011,Saturday,50437281700.268036,1986862
